<a href="https://colab.research.google.com/github/catalinamagalvan/PySimMIBCI/blob/main/Load_data_ftp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from scipy.io import loadmat
import os
import urllib.request

In [ ]:
url = 'ftp://parrot.genomics.cn/gigadb/pub/10.5524/100001_101000/100542/session1/s1/sess01_subj01_EEG_MI.mat'
urllib.request.urlretrieve(url, 'EEG_MI.mat')
data = loadmat('EEG_MI.mat')

In [8]:
!git clone https://github.com/catalinamagalvan/PySimMIBCI.git

Cloning into 'PySimMIBCI'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.


ModuleNotFoundError: ignored

In [10]:
import sys
sys.path.insert(1, os.path.join('PySimMIBCI', 'utils'))

In [13]:
from datasets import raw_from_OpenBMI_data

In [12]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 76.5 MB/s eta 0:00:00
